##### Author: Praveen Saxena
##### Email: saxep01@gmail.com
##### Create Date: 7/16/2021
##### Purpose: Validate the need for certain indexes on _nanohub.jos_users_ & _nanohub_metrics.userlogin_.

----

Some preliminaries.

In [5]:
import pandas as pd
from pprint import pprint
from IPython.display import display, Markdown

In [6]:
import os

print(os.getenv('APP_DIR'))

/home/nanoHUB-user/app


In [7]:
%%capture 

from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')

ModuleNotFoundError: No module named 'nanoHUB'

We start with a basic query that finds all the users except for the one-day users.
One-day users are users who registered with nanoHUB but never logged in after their first 24 hours of registration.

In [3]:
sql = '''
SELECT registerDate, lastvisitDate
    FROM nanohub.jos_users
    WHERE registerDate <= DATE_SUB(lastvisitDate, INTERVAL 1 DAY)
LIMIT 100;
'''
df = pd.read_sql(sql, nanohub_db)
df.head()

NameError: name 'nanohub_db' is not defined

Let's run a _EXPLAIN_ query on this.

In [ ]:
sql = '''
EXPLAIN SELECT registerDate, lastvisitDate
    FROM nanohub.jos_users
    WHERE registerDate <= DATE_SUB(lastvisitDate, INTERVAL 1 DAY)
LIMIT 100;
'''
df = pd.read_sql(sql, nanohub_db)
df.head()

We can see that the query uses the much maligned _all_ type. This, and the fact that the query plan includes all the 263425 rows indicates the lack of appropriate indexes. In this case, the needed index would be on _registerDate_ and possibly _lastvisitDate_.

-----

Let's take it up a notch. This time we will execute a query to find the _FREQUENCY_ of user visits as the average number of times a user performed an action over his/her lifetime on nanoHUB (now - registerDate). We can use their last day on nanoHUB instead of now() but for our purposes, we will use now().

In [ ]:

sql = '''
EXPLAIN SELECT DISTINCT users.id, users.name, users.registerDate,
        COUNT(action) AS `num_actions`, COUNT(DISTINCT DATE(datetime)) as `distinct_days`,
        DATEDIFF (NOW(), users.registerDate)/COUNT(DISTINCT DATE(datetime)) as Frequency
    FROM nanohub.jos_users AS users
        LEFT JOIN nanohub_metrics.userlogin as logins ON logins.uidnumber = users.id
        WHERE logins.uidnumber != 0 
            && logins.uidnumber IS NOT NULL
            &&  registerDate <= DATE_SUB(lastvisitDate, INTERVAL 1 DAY)
LIMIT 10
;
'''

df = pd.read_sql(sql, nanohub_db)
df

We can see again the need for indexes as the entire index tree is scanned to find matching rows. The needed index would be on _nanohub_metrics.userlogin.uidnumber_.

Let's see which columns on _nanohub_metrics.userlogin_ do have an index.

In [ ]:
sql = '''
SHOW INDEX FROM nanohub_metrics.userlogin; 
'''

df = pd.read_sql(sql, nanohub_db)
df

So _userlogin_ has a multi-column index on _datetime, user, uidnumber, ip,_ and _action_ because we want to make sure that specific combination of column values is unique. Unfortunately, indexes work like a yellowbook. A yellowbook is essentially indexed by lastname, firstname. This is really great when we need to find someone by their last name first. But what if we had to find people by their first names, first?

So having a multi-column index that includes _uidnumber_ as the 3rd column doesn't help us. We need a separate index on _uidnumber_.

-----

The conclusion is that we desperately need indexes on:

1. nanohub_metrics.userlogin.uidnumber
2. nanohub_metrics.userlogin.datetime
3. nanohub.jos_users.registerDate
4. nanohub.jos_users.lastvisitDate

----